In [1]:
import anndata
import numpy as np
import sys
import torch
sys.path.append('../../../../')
import velovae as vv

In [2]:
import scvelo as scv
dataset = 'Dentategyrus'
#adata = scv.datasets.dentategyrus(file_path='data/download/Dentategyrus.h5ad')
adata = anndata.read_h5ad('data/Dentategyrus_pp.h5ad')

In [ ]:
#vv.preprocess(adata, n_gene=2000, min_shared_counts=20)
#adata.write_h5ad(f'data/{dataset}.h5ad')

In [3]:
model_path_base = f'checkpoints/{dataset}'
figure_path_base = f'figures/{dataset}'
data_path = f'velovae/continuous/{dataset}'
gene_plot = ['Tmsb10', 'Fam155a', 'Hn1', 'Rpl6']

# Vanilla VAE

In [4]:
figure_path = f'{figure_path_base}/Vanilla'
model_path = f'{model_path_base}/Vanilla'

torch.manual_seed(2022)
np.random.seed(2022)

vanilla_vae = vv.VanillaVAE(adata, 
                            tmax=20, 
                            device='cuda:0',
                            init_method='steady')

vanilla_vae.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)
vanilla_vae.save_model(model_path, 'encoder', 'decoder')
vanilla_vae.save_anndata(adata, 'vanilla', data_path, file_name=f'{dataset}.h5ad')

Initialization using the steady-state and dynamical models.
Estimating ODE parameters...


  0%|          | 0/2000 [00:00<?, ?it/s]

Estimating the variance...


  0%|          | 0/2000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/2000 [00:00<?, ?it/s]

Gaussian Prior.
------------------------- Train a Vanilla VAE -------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
Total Number of Iterations Per Epoch: 17, test iteration: 32
********* Early Stop Triggered at epoch 49. *********
*********              Finished. Total Time =   0 h :  0 m : 27 s             *********
Final: Train ELBO = -267.725,           Test ELBO = -244.685


# VeloVAE

In [5]:
figure_path = f'{figure_path_base}/VeloVAE'
model_path = f'{model_path_base}/VeloVAE'

torch.manual_seed(2022)
np.random.seed(2022)
vae = vv.VAE(adata, 
             tmax=20, 
             dim_z=5, 
             device='cuda:0',
             init_method='steady')

vae.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

vae.save_model(model_path, 'encoder', 'decoder')
vae.save_anndata(adata, 'velovae', data_path, file_name=f'{dataset}.h5ad')

Initialization using the steady-state and dynamical models.
Estimating ODE parameters...


  0%|          | 0/2000 [00:00<?, ?it/s]

Estimating the variance...


  0%|          | 0/2000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/2000 [00:00<?, ?it/s]

Gaussian Prior.
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 17, test iteration: 32
*********       Stage 1: Early Stop Triggered at epoch 633.       *********
*********                      Stage  2                       *********
Cell-wise KNN Estimation.


  0%|          | 0/2930 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.038
Average Set Size: 82
Finished. Actual Time:   0 h :  0 m :  2 s
*********       Stage 2: Early Stop Triggered at epoch 730.       *********
*********              Finished. Total Time =   0 h :  2 m : 43 s             *********
Final: Train ELBO = 2905.851,           Test ELBO = 2930.647


# Full VB

In [6]:
figure_path = f'{figure_path_base}/FullVB'
model_path = f'{model_path_base}/FullVB'

torch.manual_seed(2022)
np.random.seed(2022)
full_vb = vv.VAEFullVB(adata, 
                       tmax=20, 
                       dim_z=5, 
                       device='cuda:0',
                       init_method='steady')

full_vb.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

full_vb.save_model(model_path, 'encoder', 'decoder')
full_vb.save_anndata(adata, 'fullvb', data_path, file_name=f'{dataset}.h5ad')

Initialization using the steady-state and dynamical models.
Estimating ODE parameters...


  0%|          | 0/2000 [00:00<?, ?it/s]

Estimating the variance...


  0%|          | 0/2000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/2000 [00:00<?, ?it/s]

Gaussian Prior.
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 17, test iteration: 32
*********                      Stage  2                       *********
Cell-wise KNN Estimation.


  0%|          | 0/2930 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.061
Average Set Size: 100
Finished. Actual Time:   0 h :  0 m :  2 s
*********       Stage 2: Early Stop Triggered at epoch 1072.       *********
*********              Finished. Total Time =   0 h :  4 m : 26 s             *********
Final: Train ELBO = 2986.635,           Test ELBO = 2938.056


# Branching ODE

In [7]:
figure_path = f'{figure_path_base}/BrODE'
model_path = f'{model_path_base}/BrODE'

torch.manual_seed(2022)
np.random.seed(2022)
brode = vv.BrODE(adata, 'clusters', 'fullvb_time', 'fullvb_z')

brode.train(adata, 'fullvb_time', 'clusters', plot=False, gene_plot=gene_plot, figure_path=figure_path)

brode.save_model(model_path, 'brode')
brode.save_anndata(adata, 'brode', data_path, file_name=f'{dataset}.h5ad')

vv.plot_transition_graph(adata, save=f'{figure_path_base}/transition.png')

Graph Partition
Number of partitions:  6
Computing type-to-type transition probability
Obtaining the MST in each partition
Initialization using type-specific dynamical model.
Estimating ODE parameters...


  0%|          | 0/2000 [00:00<?, ?it/s]

Estimating the variance...


  0%|          | 0/2000 [00:00<?, ?it/s]

------------------------ Train a Branching ODE ------------------------
                  Astrocytes  Cajal Retzius  Cck-Tox  Endothelial  GABA  Granule immature  Granule mature  Microglia  Mossy  Neuroblast   OL  OPC  Radial Glia-like  nIPC
Astrocytes               0.0            0.0      0.0          0.0   0.0               0.0             0.0        0.0    0.0         0.0  0.0  0.0               1.0   0.0
Cajal Retzius            0.0            1.0      0.0          0.0   0.0               0.0             0.0        0.0    0.0         0.0  0.0  0.0               0.0   0.0
Cck-Tox                  0.0            0.0      0.0          0.0   1.0               0.0             0.0        0.0    0.0         0.0  0.0  0.0               0.0   0.0
Endothelial              0.0            0.0      0.0          1.0   0.0               0.0             0.0        0.0    0.0         0.0  0.0  0.0               0.0   0.0
GABA                     0.0            0.0      0.0          0.0   0.0       

# Evaluation

In [ ]:
cluster_edges = [('OPC', 'OL')]
vv.post_analysis(adata,
                 'eval',
                 ['VeloVAE','FullVB','BrODE'],
                 ['velovae','fullvb','brode'],
                 compute_metrics=True,
                 genes=gene_plot,
                 grid_size=(1,4),
                 plot_type=['all'],
                 save_path=data_path,
                 cluster_edges=cluster_edges)